In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import linregress
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

TICKER = 'ADP'

print(f"Downloading daily data for {TICKER} from January 1, 2019...")
data_daily = yf.download(TICKER, start='2019-01-01', interval='1d', progress=False)

if isinstance(data_daily.columns, pd.MultiIndex):
    data_daily.columns = data_daily.columns.droplevel(1)

# Resample to weekly, ending on Fridays
data = data_daily.resample('W-FRI').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': 'sum'
}).dropna()

print(f"Total weeks: {len(data)}")
print(f"Range: {data.index[0].date()} to {data.index[-1].date()}")
print(f"\nLast few rows:")
print(data.tail())

In [ ]:
"""
STEP 1: Analyze most recent 10 weeks to generate X-Y-D/U pattern notation.
"""

current_window = data.tail(10).copy()

# Count up/down weeks (Close vs prior Close)
current_window['Up'] = (current_window['Close'] > current_window['Close'].shift(1)).astype(int)
up_weeks = current_window['Up'].sum()
down_weeks = 10 - up_weeks

# Calculate trajectory
close_prices = current_window['Close'].values.flatten()
weeks_index = np.arange(len(close_prices))
slope, intercept, r_value, p_value, std_err = linregress(weeks_index, close_prices)
trajectory = 'U' if slope > 0 else 'D'

# Entry price = closing price at END of pattern
entry_price = float(current_window.iloc[-1]['Close'].iloc[0] if hasattr(current_window.iloc[-1]['Close'], 'iloc') else current_window.iloc[-1]['Close'])

current_pattern = f"{up_weeks}-{down_weeks}-{trajectory}"

print("=" * 60)
print("STEP 1: CURRENT 10-WEEK SEQUENCE")
print("=" * 60)
print(f"Pattern: {current_pattern}")
print(f"Up Weeks: {up_weeks}")
print(f"Down Weeks: {down_weeks}")
print(f"Trajectory: {trajectory} (slope: {slope:.4f})")
print(f"Entry Price: ${entry_price:.2f}")
print(f"Period: {current_window.index[0].date()} to {current_window.index[-1].date()}")

In [ ]:
"""
STEP 2: Scan historical data for matching pattern instances.
"""

def calculate_pattern(window_df):
    if len(window_df) < 10:
        return None
    up = (window_df['Close'] > window_df['Close'].shift(1)).sum()
    down = 10 - up
    closes = window_df['Close'].values.flatten()
    idx = np.arange(len(closes))
    slope, _, _, _, _ = linregress(idx, closes)
    traj = 'U' if slope > 0 else 'D'
    return f"{up}-{down}-{traj}"

patterns_list = []

for i in range(len(data) - 9):
    window = data.iloc[i:i+10]
    pattern = calculate_pattern(window)
    if pattern:
        pattern_end_price = float(window.iloc[-1]['Close'].iloc[0] if hasattr(window.iloc[-1]['Close'], 'iloc') else window.iloc[-1]['Close'])
        patterns_list.append({
            'pattern': pattern,
            'start_date': window.index[0],
            'end_date': window.index[-1],
            'pattern_end_price': pattern_end_price,
            'window_idx': i
        })

patterns_df = pd.DataFrame(patterns_list)
matches = patterns_df[patterns_df['pattern'] == current_pattern]
total_patterns = len(patterns_df)
match_count = len(matches)
frequency = (match_count / total_patterns) * 100 if total_patterns > 0 else 0

# ENOMOTO METHODOLOGY: Use only last 10 matches (L10) for pattern-specific analysis
matches_l10 = matches.tail(10)
l10_count = len(matches_l10)

print("=" * 60)
print("STEP 2: HISTORICAL PATTERN OCCURRENCES")
print("=" * 60)
print(f"Current Pattern: {current_pattern}")
print(f"Total Historical Windows: {total_patterns}")
print(f"Total Matching Patterns: {match_count}")
print(f"L10 Matches (Last 10): {l10_count}")
print(f"Frequency: {frequency:.2f}%")
print(f"Rarity: {'Common' if frequency > 5 else 'Moderate' if frequency > 2 else 'Rare'}")

if match_count > 0:
    print(f"\nSample matches (showing all if ≤10, or last 10):")
    print(matches_l10[['start_date', 'end_date', 'pattern_end_price']])
else:
    print(f"\n⚠️  No matches found - analysis will use baseline only")

In [ ]:
"""
STEP 3 & 4 (ENOMOTO BARCHART METHODOLOGY): 
Calculate baseline and L10 pattern-specific distributions, then create bimodal GMM on combined data.

KEY ENOMOTO METHODOLOGY:
1. Use ALL weeks 1-10 instead of just week 10
2. Use ACTUAL historical prices instead of projecting onto entry_price
3. Use only LAST 10 MATCHES (L10) for pattern-specific analysis
4. Create BIMODAL GMM on COMBINED baseline + L10 data
5. Barchart-style visualization with blue baseline, green L10, and red GMM curve
"""

FORWARD_WEEKS = 10

def get_price_clustering_gmm(prices_array, n_components_range=(1, 4)):
    """
    Uses GMM with BIC comparison to find optimal clustering.
    Returns the dominant cluster price (highest weight component).
    """
    if len(prices_array) < 20:
        if len(prices_array) > 0:
            return np.median(prices_array), 1, None
        else:
            return np.nan, 0, None
    
    X = prices_array.reshape(-1, 1)
    
    best_gmm = None
    best_bic = np.inf
    
    for n_components in range(n_components_range[0], n_components_range[1]):
        gmm = GaussianMixture(n_components=n_components, random_state=42)
        gmm.fit(X)
        bic = gmm.bic(X)
        if bic < best_bic:
            best_bic = bic
            best_gmm = gmm
    
    # Find dominant cluster (highest weight)
    dominant_idx = np.argmax(best_gmm.weights_)
    cluster_price = best_gmm.means_[dominant_idx][0]
    
    return cluster_price, best_gmm.n_components, best_gmm

# === BASELINE: Collect ALL future prices from ALL patterns ===
baseline_future_prices = []

for idx, row in patterns_df.iterrows():
    window_end_idx = row['window_idx'] + 9  # Last week of pattern
    
    # Collect ALL prices from weeks 1-10 ahead (not just week 10)
    for week_offset in range(1, FORWARD_WEEKS + 1):
        future_idx = window_end_idx + week_offset
        if future_idx < len(data):
            future_price = data.iloc[future_idx]['Close']
            baseline_future_prices.append(float(future_price.iloc[0] if hasattr(future_price, 'iloc') else future_price))

baseline_prices_array = np.array(baseline_future_prices)
baseline_median = np.median(baseline_prices_array)

print("=" * 60)
print("STEP 3: BASELINE EXPECTATIONS (GLOBAL)")
print("=" * 60)
print(f"Sample Size: {len(baseline_prices_array)} price points")
print(f"  ({len(patterns_df)} patterns × {FORWARD_WEEKS} weeks)")
print(f"Forward Period: {FORWARD_WEEKS} weeks")
print(f"\nBaseline Statistics (using ACTUAL historical prices):")
print(f"  Mean: ${baseline_prices_array.mean():.2f}")
print(f"  Global Median: ${baseline_median:.2f}")

# === PATTERN-SPECIFIC L10: Collect ALL future prices from LAST 10 MATCHING patterns ===
pattern_l10_future_prices = []

for idx, row in matches_l10.iterrows():
    window_end_idx = row['window_idx'] + 9
    
    # Collect ALL prices from weeks 1-10 ahead
    for week_offset in range(1, FORWARD_WEEKS + 1):
        future_idx = window_end_idx + week_offset
        if future_idx < len(data):
            future_price = data.iloc[future_idx]['Close']
            pattern_l10_future_prices.append(float(future_price.iloc[0] if hasattr(future_price, 'iloc') else future_price))

if len(pattern_l10_future_prices) > 0:
    pattern_l10_prices_array = np.array(pattern_l10_future_prices)
    pattern_l10_median = np.median(pattern_l10_prices_array)
    
    print("\n" + "=" * 60)
    print("STEP 4: L10 PATTERN-SPECIFIC DISTRIBUTION")
    print("=" * 60)
    print(f"Pattern: {current_pattern}")
    print(f"Sample Size: {len(pattern_l10_prices_array)} price points")
    print(f"  ({l10_count} patterns × {FORWARD_WEEKS} weeks)")
    print(f"\nL10 Pattern Statistics (using ACTUAL historical prices):")
    print(f"  Mean: ${pattern_l10_prices_array.mean():.2f}")
    print(f"  L10 Median: ${pattern_l10_median:.2f}")
    
    # === ENOMOTO BIMODAL GMM: Fit 2-component GMM on COMBINED data ===
    combined_prices = np.concatenate([baseline_prices_array, pattern_l10_prices_array])
    bimodal_gmm = GaussianMixture(n_components=2, random_state=42)
    bimodal_gmm.fit(combined_prices.reshape(-1, 1))
    
    # Extract the two means
    gmm_mean_1 = bimodal_gmm.means_[0][0]
    gmm_mean_2 = bimodal_gmm.means_[1][0]
    
    print("\n" + "=" * 60)
    print("BIMODAL GMM ON COMBINED DATA")
    print("=" * 60)
    print(f"Combined Sample: {len(combined_prices)} price points")
    print(f"  (Baseline: {len(baseline_prices_array)} + L10: {len(pattern_l10_prices_array)})")
    print(f"GMM Component 1 Mean: ${gmm_mean_1:.2f}")
    print(f"GMM Component 2 Mean: ${gmm_mean_2:.2f}")
    
    # === BARCHART-STYLE VISUALIZATION ===
    plt.figure(figsize=(14, 7))
    
    # Plot baseline KDE as blue filled area
    sns.kdeplot(data=baseline_prices_array, fill=True, label='All outcomes', 
                alpha=0.3, color='blue')
    
    # Plot L10 pattern KDE as green filled area
    pattern_label = f'L10 ({current_pattern})'
    sns.kdeplot(data=pattern_l10_prices_array, fill=True, label=pattern_label,
                alpha=0.5, color='green')
    
    # Plot bimodal GMM as RED CURVE
    x_min = min(combined_prices.min(), baseline_prices_array.min(), pattern_l10_prices_array.min())
    x_max = max(combined_prices.max(), baseline_prices_array.max(), pattern_l10_prices_array.max())
    x_range = np.linspace(x_min, x_max, 1000).reshape(-1, 1)
    
    # Calculate GMM probabilities using score_samples
    log_prob = bimodal_gmm.score_samples(x_range)
    prob = np.exp(log_prob)
    
    # Plot as smooth red curve
    plt.plot(x_range, prob, color='red', linewidth=2, label='Bimodal Fit (GMM)')
    
    # Vertical lines for medians and entry price
    plt.axvline(baseline_median, color='blue', linestyle='--', linewidth=1.5,
                label=f'Global Median: ${baseline_median:.2f}')
    plt.axvline(pattern_l10_median, color='green', linestyle='--', linewidth=1.5,
                label=f'L10 Median: ${pattern_l10_median:.2f}')
    plt.axvline(entry_price, color='black', linestyle='-', linewidth=2,
                label=f'Entry Price: ${entry_price:.2f}')
    
    plt.xlabel(f'Price (Weeks 1-{FORWARD_WEEKS} ahead)', fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.title(f'L10 Median vs Global Median - {current_pattern}\n' +
              f'Baseline: {len(baseline_prices_array)} prices vs L10: {len(pattern_l10_prices_array)} prices',
              fontsize=14, fontweight='bold')
    plt.legend(loc='upper right', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
else:
    pattern_l10_median = np.nan
    pattern_l10_prices_array = np.array([])
    bimodal_gmm = None
    print("\nNo L10 pattern matches found - cannot calculate pattern-specific distribution.")

In [ ]:
"""
STEP 5: Calculate delta and exceedance metrics (ENOMOTO L10 METHODOLOGY).
"""

if len(pattern_l10_future_prices) > 0 and not np.isnan(pattern_l10_median):
    # Delta based on L10 median vs global median
    delta_pct = ((pattern_l10_median - baseline_median) / baseline_median) * 100
    
    exceedance_count = (pattern_l10_prices_array > entry_price).sum()
    exceedance_ratio = (exceedance_count / len(pattern_l10_prices_array)) * 100
    
    terminal_median_price = pattern_l10_median
    
    print("=" * 60)
    print("STEP 5: DELTA IN PRICE DENSITY DYNAMICS (L10)")
    print("=" * 60)
    print(f"Global Median: ${baseline_median:.2f}")
    print(f"L10 Median: ${pattern_l10_median:.2f}")
    print(f"Delta: {delta_pct:+.2f}%")
    print(f"Signal: {'BULLISH - Positive delta' if delta_pct > 0 else 'BEARISH - Negative delta'}")
    
    print("\n" + "=" * 60)
    print("STEP 6: SUPPORTING METRICS")
    print("=" * 60)
    print(f"Entry Price: ${entry_price:.2f}")
    print(f"L10 Exceedance Ratio: {exceedance_ratio:.1f}%")
    print(f"L10 Terminal Median: ${terminal_median_price:.2f}")
    print(f"L10 Sample Size: {len(pattern_l10_prices_array)} price points from {l10_count} pattern instances")
    
    # Compare to baseline statistics
    baseline_exceedance = (baseline_prices_array > entry_price).sum()
    baseline_exceedance_ratio = (baseline_exceedance / len(baseline_prices_array)) * 100
    print(f"\nComparison to Baseline:")
    print(f"  Baseline Exceedance Ratio: {baseline_exceedance_ratio:.1f}%")
    print(f"  L10 Exceedance Ratio: {exceedance_ratio:.1f}%")
    print(f"  Difference: {exceedance_ratio - baseline_exceedance_ratio:+.1f}%")
    
    METRICS = {
        'delta_pct': delta_pct,
        'exceedance_ratio': exceedance_ratio,
        'pattern_l10_median': pattern_l10_median,
        'terminal_median_price': terminal_median_price,
        'baseline_median': baseline_median,
        'pattern_l10_mean': pattern_l10_prices_array.mean(),
        'baseline_mean': baseline_prices_array.mean(),
        'l10_count': l10_count,
        'l10_sample_size': len(pattern_l10_prices_array)
    }
else:
    METRICS = None
    print("\nNo L10 pattern matches - cannot calculate delta.")

In [ ]:
"""
STEP 7: Design bull call spread based on positive delta exploitation (ENOMOTO L10 METHODOLOGY).
"""

if METRICS and METRICS['delta_pct'] > 0:
    last_date = data.index[-1]
    expiration_date = last_date + timedelta(weeks=FORWARD_WEEKS)
    
    # Strike selection: center on L10 median
    pattern_l10_median_price = METRICS['pattern_l10_median']
    
    # Short strike targets the L10 median (where price likely lands)
    short_strike = round(pattern_l10_median_price * 2) / 2  # Round to $0.50
    
    # Long strike 5-10 points below for spread width
    spread_width = 10 if entry_price > 200 else 5
    long_strike = short_strike - spread_width
    
    # Alternative conservative spread using baseline
    conservative_long = round(METRICS['baseline_median'] * 2) / 2
    conservative_short = conservative_long + spread_width
    
    upside_to_l10 = ((pattern_l10_median_price - entry_price) / entry_price) * 100
    
    print("=" * 60)
    print(f"STEP 7: BULL CALL SPREAD RECOMMENDATION - {TICKER} (ENOMOTO L10)")
    print("=" * 60)
    print(f"\n📊 PATTERN: {current_pattern}")
    print(f"   Total Historical Matches: {match_count}")
    print(f"   L10 Matches Used: {l10_count}")
    print(f"   Frequency: {frequency:.2f}% ({'Ultra-rare' if frequency < 3 else 'Rare' if frequency < 5 else 'Moderate'})")
    
    print(f"\n📈 KEY METRICS (ENOMOTO L10 METHODOLOGY):")
    print(f"   Positive Delta: +{METRICS['delta_pct']:.2f}%")
    print(f"   Global Median: ${METRICS['baseline_median']:.2f}")
    print(f"   L10 Median: ${pattern_l10_median_price:.2f}")
    print(f"   L10 Exceedance Ratio: {METRICS['exceedance_ratio']:.1f}%")
    print(f"   L10 Sample: {METRICS['l10_sample_size']} prices from {l10_count} instances")
    
    print(f"\n💰 PRIMARY STRATEGY - AGGRESSIVE:")
    print(f"   Long Strike:  ${long_strike:.2f} (BUY)")
    print(f"   Short Strike: ${short_strike:.2f} (SELL)")
    print(f"   Spread Width: ${spread_width:.2f}")
    print(f"   Max Profit: ${spread_width:.2f} per contract")
    print(f"   Expiration: {expiration_date.strftime('%Y-%m-%d')}")
    print(f"   Target: ${pattern_l10_median_price:.2f} ({upside_to_l10:+.1f}% from entry)")
    
    print(f"\n💡 ALTERNATIVE - CONSERVATIVE:")
    print(f"   Long Strike:  ${conservative_long:.2f} (BUY)")
    print(f"   Short Strike: ${conservative_short:.2f} (SELL)")
    print(f"   Rationale: Lower strikes for higher probability of profit")
    
    print(f"\n🎯 PRICE LEVELS:")
    print(f"   Current Entry: ${entry_price:.2f}")
    print(f"   L10 Target: ${pattern_l10_median_price:.2f}")
    print(f"   L10 Terminal Median: ${METRICS['terminal_median_price']:.2f}")
    print(f"   Global Baseline: ${METRICS['baseline_median']:.2f}")
    
    print(f"\n📊 EDGE CALCULATION:")
    print(f"   L10 shows {METRICS['delta_pct']:.1f}% median improvement over global baseline")
    print(f"   Historical probability: ~{METRICS['exceedance_ratio']:.0f}% of L10 reaching above entry")
    print(f"   Exploits positive delta in L10 price density dynamics")
    
    print("\n" + "=" * 60)
    print("TRADE MANAGEMENT:")
    print("• Enter when spread offers 2:1+ reward:risk")
    print("• Target 50-80% max profit (close early)")
    print("• Stop loss: -50% of debit paid")
    print("• Monitor for pattern invalidation")
    print("=" * 60)
    
    print("\n" + "=" * 60)
    print("ENOMOTO BARCHART METHODOLOGY APPLIED:")
    print("=" * 60)
    print("✓ Uses LAST 10 MATCHES (L10) for pattern-specific analysis")
    print("✓ Uses ALL weeks 1-10 (not just week 10)")
    print("✓ Uses ACTUAL historical prices (no projection bias)")
    print("✓ Creates BIMODAL GMM on combined baseline + L10 data")
    print("✓ Barchart-style visualization: blue baseline, green L10, red GMM")
    print(f"✓ L10 sample: {METRICS['l10_sample_size']} prices from {l10_count} instances")
    print("=" * 60)
    
elif METRICS and METRICS['delta_pct'] < 0:
    print("=" * 60)
    print(f"NO BULLISH TRADE - Negative Delta")
    print("=" * 60)
    print(f"L10 shows {METRICS['delta_pct']:.2f}% negative delta")
    print(f"L10 median (${METRICS['pattern_l10_median']:.2f}) below global median (${METRICS['baseline_median']:.2f})")
    print("\nConsider: Bearish strategies or avoid this trade")
    
else:
    print("\n⚠️  Insufficient L10 pattern data for strategy recommendation")